In [ ]:
# import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib qt 

In [2]:
collar = pd.read_csv("C:/Users/TIYANI/Videos/Final/Collar.csv")

In [3]:
collar.head()

,Hole_ID,X,Y,Z,Depth_max,target
0,KRED207,530167.060,8714340.880,1330.600,104.5,1
1,KRED209,530024.162,8714201.452,1331.309,113.0,1
2,KRED211,530079.988,8714416.366,1330.504,155.0,1
3,KRED212,530246.989,8714518.866,1329.942,161.0,1
4,KRED218,530027.489,8714330.724,1331.295,170.0,1


In [4]:
collar.shape

(66, 6)

In [5]:
collar.describe()

,X,Y,Z,Depth_max,target
count,66.000000,6.600000e+01,66.000000,66.000000,66.000000
mean,530120.498545,8.714424e+06,1330.574621,148.265152,1.121212
std,131.854344,1.394638e+02,0.914364,45.392919,0.328875
min,529863.836000,8.714125e+06,1328.925000,77.000000,1.000000
25%,530027.749500,8.714330e+06,1329.912250,116.625000,1.000000
50%,530123.005500,8.714419e+06,1330.555000,144.750000,1.000000
75%,530218.644500,8.714519e+06,1331.289500,170.000000,1.000000
max,530397.278000,8.714717e+06,1332.649000,287.000000,2.000000


In [6]:
collar.dtypes

Hole_ID       object
X            float64
Y            float64
Z            float64
Depth_max    float64
target         int64
dtype: object

In [7]:
collar.isna().any()

Hole_ID      False
X            False
Y            False
Z            False
Depth_max    False
target       False
dtype: bool

In [8]:
collar.columns

Index(['Hole_ID', 'X', 'Y', 'Z', 'Depth_max', 'target'], dtype='object')

In [9]:
from sdv.constraints import FixedCombinations

In [10]:
Location = FixedCombinations(column_names=['X', 'Y', 'Z', 'Depth_max'])

In [ ]:
Depth=FixedCombinations(column_names=['Z', 'Depth_max'])

In [11]:
constraints = [Location]

In [12]:
from sdv.tabular import CTGAN

In [13]:
batch_size = 2000
epochs = 100

model = CTGAN(primary_key='Hole_ID',constraints=constraints, generator_dim=(257, 257, 257),
              discriminator_dim=(257, 257, 257), batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(collar)

Epoch 1, Loss G:  4.2787,Loss D:  0.0013
Epoch 2, Loss G:  4.2475,Loss D:  0.0031
Epoch 3, Loss G:  4.2403,Loss D:  0.0012
Epoch 4, Loss G:  4.2197,Loss D: -0.0010
Epoch 5, Loss G:  4.2015,Loss D: -0.0029
Epoch 6, Loss G:  4.1942,Loss D: -0.0063
Epoch 7, Loss G:  4.1782,Loss D:  0.0030
Epoch 8, Loss G:  4.1867,Loss D: -0.0005
Epoch 9, Loss G:  4.1596,Loss D: -0.0132
Epoch 10, Loss G:  4.1620,Loss D: -0.0049
Epoch 11, Loss G:  4.1590,Loss D: -0.0081
Epoch 12, Loss G:  4.1414,Loss D: -0.0043
Epoch 13, Loss G:  4.1112,Loss D: -0.0039
Epoch 14, Loss G:  4.1022,Loss D:  0.0034
Epoch 15, Loss G:  4.0830,Loss D: -0.0015
Epoch 16, Loss G:  4.0502,Loss D:  0.0097
Epoch 17, Loss G:  4.0514,Loss D:  0.0127
Epoch 18, Loss G:  4.0238,Loss D:  0.0149
Epoch 19, Loss G:  3.9906,Loss D:  0.0358
Epoch 20, Loss G:  3.9986,Loss D:  0.0290
Epoch 21, Loss G:  3.9579,Loss D:  0.0193
Epoch 22, Loss G:  3.9897,Loss D:  0.0257
Epoch 23, Loss G:  3.9879,Loss D:  0.0174
Epoch 24, Loss G:  3.9676,Loss D:  0.0225
E

In [14]:
n_generated_data = 66
generated_df = model.sample(n_generated_data)

Sampling rows: 100%|██████████| 66/66 [00:00<00:00, 191.80it/s]


In [15]:
from sdv.evaluation import evaluate

In [16]:
generated_df.head()

,Hole_ID,X,Y,Z,Depth_max,target
0,a,530024.162,8714201.452,1331.309,113.0,2
1,b,530123.954,8714382.805,1330.650,132.0,1
2,c,530157.823,8714464.460,1330.200,123.5,1
3,d,530159.801,8714664.869,1329.711,171.5,1
4,e,530035.861,8714458.178,1330.968,272.0,1


In [17]:
score = evaluate(generated_df, collar.sample(n_generated_data))
score

0.45151515151515154

In [18]:
from table_evaluator import load_data, TableEvaluator

In [19]:
from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(collar, generated_df)
table_evaluator.evaluate(target_col='target')


Classifier F1-scores and their Jaccard similarities::
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.6429   0.6429              0.4000
DecisionTreeClassifier_real   0.9286   0.4286              0.2174
LogisticRegression_fake       0.9286   0.9286              1.0000
LogisticRegression_real       0.7143   0.7143              1.0000
MLPClassifier_fake            0.9286   0.9286              1.0000
MLPClassifier_real            0.7143   0.7143              1.0000
RandomForestClassifier_fake   0.6429   0.7143              0.4737
RandomForestClassifier_real   0.8571   0.7857              0.6471

Privacy results:
                                         result
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.3834
nearest neighbor std                     0.3409

Miscellaneous results:
                                  Result
Column Cor

In [20]:
table_evaluator.visual_evaluation()

In [21]:
from sdv.tabular import CopulaGAN

In [22]:
model = CopulaGAN(primary_key='Hole_ID',
                 constraints=constraints,epochs=100,batch_size=5000,
                  generator_dim=(256, 256, 256),discriminator_dim=(256, 256, 256),verbose=True)
model.fit(collar)
new_data = model.sample(66)
new_data.head()

Epoch 1, Loss G:  4.2822,Loss D: -0.0007
Epoch 2, Loss G:  4.2573,Loss D: -0.0010
Epoch 3, Loss G:  4.2449,Loss D: -0.0003
Epoch 4, Loss G:  4.2514,Loss D: -0.0029
Epoch 5, Loss G:  4.2299,Loss D: -0.0054
Epoch 6, Loss G:  4.2136,Loss D: -0.0039
Epoch 7, Loss G:  4.2077,Loss D: -0.0060
Epoch 8, Loss G:  4.1932,Loss D: -0.0082
Epoch 9, Loss G:  4.1883,Loss D: -0.0064
Epoch 10, Loss G:  4.1648,Loss D: -0.0044
Epoch 11, Loss G:  4.1525,Loss D: -0.0041
Epoch 12, Loss G:  4.1359,Loss D:  0.0027
Epoch 13, Loss G:  4.1164,Loss D:  0.0051
Epoch 14, Loss G:  4.0883,Loss D:  0.0095
Epoch 15, Loss G:  4.0655,Loss D:  0.0138
Epoch 16, Loss G:  4.0501,Loss D:  0.0128
Epoch 17, Loss G:  4.0328,Loss D:  0.0026
Epoch 18, Loss G:  4.0242,Loss D:  0.0024
Epoch 19, Loss G:  4.0114,Loss D:  0.0042
Epoch 20, Loss G:  3.9983,Loss D: -0.0066
Epoch 21, Loss G:  4.0027,Loss D: -0.0107
Epoch 22, Loss G:  3.9910,Loss D: -0.0149
Epoch 23, Loss G:  3.9913,Loss D: -0.0312
Epoch 24, Loss G:  3.9781,Loss D: -0.0428
E

Sampling rows: 100%|██████████| 66/66 [00:00<00:00, 144.30it/s]


,Hole_ID,X,Y,Z,Depth_max,target
0,a,"530,283.1570","8,714,466.6110","1,329.8940",180.0000,1
1,b,"530,123.0080","8,714,517.5800","1,330.0000",150.0000,1
2,c,"530,128.8040","8,714,246.9610","1,330.8160",78.5000,1
3,d,"530,094.7160","8,714,213.9850","1,331.0630",101.0000,1
4,e,"530,374.9550","8,714,519.0770","1,329.4260",118.5000,1


In [23]:
from sdv.evaluation import evaluate

evaluate(new_data, collar)

0.4590909090909091

In [24]:
from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(collar, new_data)
table_evaluator.evaluate(target_col='target')


Classifier F1-scores and their Jaccard similarities::
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.7857   0.8571              0.6471
DecisionTreeClassifier_real   1.0000   0.9286              0.8667
LogisticRegression_fake       1.0000   1.0000              1.0000
LogisticRegression_real       0.8571   0.8571              1.0000
MLPClassifier_fake            1.0000   1.0000              1.0000
MLPClassifier_real            0.8571   0.8571              1.0000
RandomForestClassifier_fake   0.7857   0.9286              0.7500
RandomForestClassifier_real   1.0000   0.9286              0.8667

Privacy results:
                                         result
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.4385
nearest neighbor std                     0.2729

Miscellaneous results:
                                  Result
Column Cor

In [25]:
table_evaluator.visual_evaluation()

In [26]:
from sdv.tabular import GaussianCopula

In [27]:
model = GaussianCopula(primary_key='Hole_ID',constraints=constraints,
                      field_distributions={'Z': 'beta'})
model.fit(collar)
new_data2 = model.sample(66)
new_data2.head()

Sampling rows: 100%|██████████| 66/66 [00:00<00:00, 398.43it/s]


,Hole_ID,X,Y,Z,Depth_max,target
0,a,"530,250.7730","8,714,398.7240","1,330.3520",92.0000,1
1,b,"530,189.7420","8,714,515.0500","1,329.0000",189.0000,1
2,c,"530,128.8040","8,714,246.9610","1,330.8160",78.5000,1
3,d,"530,123.9540","8,714,382.8050","1,330.6500",132.0000,1
4,e,"530,123.0030","8,714,615.9760","1,330.0740",178.0000,1


In [28]:
from sdv.evaluation import evaluate

evaluate(new_data2, collar)

0.45757575757575764

In [29]:
from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(collar, new_data2)
table_evaluator.evaluate(target_col='target')


Classifier F1-scores and their Jaccard similarities::
                             f1_real  f1_fake  jaccard_similarity
index                                                            
DecisionTreeClassifier_fake   0.7857   0.8571              0.8667
DecisionTreeClassifier_real   1.0000   0.8571              0.7500
LogisticRegression_fake       0.8571   0.8571              1.0000
LogisticRegression_real       0.8571   0.8571              1.0000
MLPClassifier_fake            0.8571   0.8571              1.0000
MLPClassifier_real            0.8571   0.8571              1.0000
RandomForestClassifier_fake   0.7857   0.7857              0.7500
RandomForestClassifier_real   1.0000   0.8571              0.7500

Privacy results:
                                         result
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    1.4411
nearest neighbor std                     0.2493

Miscellaneous results:
                                  Result
Column Cor

In [30]:
table_evaluator.visual_evaluation()